# Corso di Probabilità e Statistica- Le proprietà statistiche dell'effetto Barkhausen

L'effetto Barkhausen è il rumore emesso da un ferromagnete durante il ciclo di isteresi. Le sue proprietà rivelano la complessità del moto delle pareti ferromagnetiche. In questo capitolo esploreremo le proprietà statistiche del rumore Barkhausen. 

I dati sperimentali sono dei segnali misurati su dei campioni di una lega policristallina FeSi da Gianfranco Durin (INRIM). Il voltaggio misurato è una misura della derivata della magnetizzazione $v=dB/dt$ 
in funzione del tempo. Gli esperimenti sono stati condotti a diverse frequenze $c= dH/dt$. Lo scopo di questo studio è di stabilire come la distribuzione del rumore dipenda dalla frequenza



In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import scipy.stats 
import scipy
import seaborn as sns
import matplotlib.pylab as plt

In [ ]:
# Leggiamo i file
FS0005=scipy.genfromtxt("../data/Barkhausen/FS78G_0.005_time_1.dat")
FS001=scipy.genfromtxt("../data/Barkhausen/FS78G_0.01_time_1.dat")
FS002=scipy.genfromtxt("../data/Barkhausen/FS78G_0.02_time_1.dat")
FS003=scipy.genfromtxt("../data/Barkhausen/FS78G_0.03_time_1.dat")
FS005=scipy.genfromtxt("../data/Barkhausen/FS78G_0.05_time_1.dat")

In [ ]:
# inseriamo i files in un unico dataframe
df=pd.DataFrame(data=FS0005,columns=["Voltage"])
df["Frequency"]=0.005
df['time'] = range(1, len(df) + 1)
df1=pd.DataFrame(data=FS001,columns=["Voltage"])
df1["Frequency"]=0.01
df1['time'] = range(1, len(df1) + 1)
df=df.append(df1)
df1=pd.DataFrame(data=FS002,columns=["Voltage"])
df1["Frequency"]=0.02
df1['time'] = range(1, len(df1) + 1)
df=df.append(df1)
df1=pd.DataFrame(data=FS003,columns=["Voltage"])
df1["Frequency"]=0.03
df1['time'] = range(1, len(df1) + 1)
df=df.append(df1)
df1=pd.DataFrame(data=FS005,columns=["Voltage"])
df1["Frequency"]=0.05
df1['time'] = range(1, len(df1) + 1)
df=df.append(df1)

**Esporazione qualitativa** Per prima cosa facciamo dei grafici dei segnali Barkhausen, calcoliamone le medie e guardiamone la distribuzione

In [ ]:
plt.rcParams.update({'font.size': 18})
# plottiamo i segnali
g = sns.FacetGrid(df, row="Frequency", height=2, aspect=4.)
g.map_dataframe(sns.lineplot, x="time", y="Voltage")
plt.ylim((0,5e-6))
plt.xlim((0,10000))
g.set_axis_labels("time", "Voltage")

In [ ]:
#confrontiamo le proprietà dei segnali
media=df.groupby("Frequency").mean()
media.reset_index(inplace=True)
media

In [ ]:
sns.scatterplot(data=media, x="Frequency", y="Voltage")
plt.ylim((1e-08,5e-07))

Proviamo a confrontare le distribuzioni con un boxplot

In [ ]:
sns.boxplot(data=df, x="Frequency", y="Voltage")
plt.ylim((0,1e-06))

Notiamo che ci sono molti "outliers" ovvero le distribuzioni hanno delle code molto lunghe. Conviene quindi considerare il logaritmo del segnale

In [ ]:
df["log(Voltage)"]=np.log(df.Voltage)
# togliamo i NaN
df=df.dropna()
sns.boxplot(data=df, x="Frequency", y="log(Voltage)")

Consideriamo ora gli istogrammi

In [ ]:
#log-lin
g=sns.histplot(data=df,x="Voltage",hue="Frequency", 
             stat="density", kde=False, palette="Set2",
             cumulative=False,  
               common_norm=False, element="step")
g.set_yscale('log')
plt.xlim((0,1e-6))
plt.ylim((1e4,1e8))
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

In [ ]:
#log-log
g=sns.histplot(data=df,x="Voltage",hue="Frequency", 
             stat="density", kde=True, palette="Set2",
             cumulative=False, log_scale=True, 
               common_norm=False, element="step")
g.set_yscale('log')
plt.xlim((1e-8,1e-5))
plt.ylim((1e-3,1))

In [ ]:
# cumulate
g=sns.histplot(data=df,x="log(Voltage)",hue="Frequency", 
             stat="density", palette="Set2",
             cumulative=True, common_norm=False, 
             element="step" , kde=True)

### Modello

Il modello ABBM (Alessandro, Beatrice, Bertotti, Montorsi) descrive il moto di una parete ferromagnetica con il moto in un potenziale random:

$$ \frac{dx}{dt} = Ct - kx + W(x)$$

dove $ct$ è proporzionale al campo magnetico, $-kx$ al campo smagnetizzante, $W(x)$ è un random walk in $x$ e rappresenta il potenziale random con correlazioni

$$ \langle (W(x)-W(x'))^2 \rangle = D|x-x'|$$

La soluzione di questo modello prevede che la distribuzione delle velocità della parete (proporzionale al voltaggio misurato sperimentalmente) segue una distribuzione gamma:

$$ p(v) = \frac{v^{c-1} e^{-v/v_0}}{\Gamma(c)} $$

con $c=C/D$ e $v_0=D/k$


Usiamo il metodo MLE per fittare il segnale sperimentale con la teoria:

In [ ]:
c, loc, v_0= scipy.stats.gamma.fit(df[df["Frequency"]==0.05].Voltage, floc=0, scale=1e-7)

In [ ]:
c,v_0

Confrontiamo il fit con l'istogramma sperimentale

In [ ]:
xmin=0
xmax=4e-6
x = np.linspace(xmin,xmax,101)
plt.hist(df[df["Frequency"]==0.05].Voltage,bins=50, density=True, label='data',
         stacked=True, log=True, histtype="step")
gg=scipy.stats.gamma(c,loc=0, scale=v_0)
plt.plot(x, gg.pdf(x), label="Fit 1", color="black")
plt.xlabel('v')
plt.ylabel('p(v)')
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

**Esercizio**

- Fittare le distribuzioni per tutte le 5 frequenze.  
- Rappresentare tutti i fit in un unica figura
- Calcolare dal fit la media della distribuzione gamma $\langle v \rangle= c v_0$ e poi plottarla in funzione della media di ciascuna distribuzione sperimentale.
- Plottare $c$ e $v_0$ in funzione della frequenza.
